## Temporal Instanton Problem Instance

Let's make up some simple numbers and make an instance of the temporal instanton problem.

In [ ]:
n = 3
Ridx = [1,2]
slack = 3

# Generate objective matrix:
Qobj = tmp